# Handwritten text recognition

In [ ]:
# For Google Colaboratory
import sys, os
if 'google.colab' in sys.modules:
    # mount google drive
    from google.colab import drive
    drive.mount('/content/gdrive')
    # find automatically the path of the folder containing "file_name" :
    file_name = 'rnn_solution.ipynb'
    import subprocess
    # path_to_file = subprocess.check_output('find . -type f -name ' + str(file_name), shell=True).decode("utf-8")
    # path_to_file = path_to_file.replace(file_name,"").replace('\n',"")
    # if previous search failed or too long, comment the previous line and simply write down manually the path below :
    path_to_file = '/content/gdrive/My Drive/CS5242 Project/Solutions/RNN Solution/'
    print(path_to_file)
    # change current path to the folder containing "file_name"
    os.chdir(path_to_file)
    !pwd

Mounted at /content/gdrive
/content/gdrive/My Drive/CS5242 Project/RNN Solution [Aristotelis]/
/content/gdrive/.shortcut-targets-by-id/1fYoPc4cTre2OE-DPuGBLLQYb7zuw8sxY/CS5242 Project/RNN Solution [Aristotelis]


In [ ]:
import torch
import torch.nn.functional as F
import torch.nn as nn
import math
import time
import utils

### With or without GPU?

In [ ]:
device= torch.device("cuda")
print(device)

cuda


In [ ]:
import numpy as np
import utils

### Prepare the dataset

In [ ]:
train_data_folder = '../../ready_data/2/'

train_data_file = train_data_folder + 'data.pt'
train_labels_file = train_data_folder + 'labels.pt'

train_data = torch.load(train_data_file) / 255
train_labels = torch.load(train_labels_file)


test_data_folder = '../../ready_data/1/'

test_data_file = test_data_folder + 'data.pt'
test_labels_file = test_data_folder + 'labels.pt'

test_data = torch.load(test_data_file) / 255
test_labels = torch.load(test_labels_file)

### Compute mean

In [ ]:
train_mean= train_data.mean()

print(train_mean)

tensor(0.9515)


### Compute standard deviation

In [ ]:
train_std= train_data.std()

print(train_std)

tensor(0.1913)


In [ ]:
from torchvision import transforms

train_transform = transforms.Compose(
                    [
                    transforms.ToPILImage(),
                    transforms.RandomAffine(degrees=20, translate=(0.1,0.1), scale=(0.9, 1.1)),
                    transforms.ColorJitter(brightness=0.2, contrast=0.2),
                    transforms.ToTensor(),
                    ])

augmented_images = []
augmented_labels = []

for tensor, label in zip(train_data, train_labels):
    
    for i in range(60):
        new_image = train_transform(1 - tensor)
        augmented_images.append(1 - new_image)
        augmented_labels.append(label)

train_augmented_data = torch.stack(augmented_images)
train_augmented_labels = torch.stack(augmented_labels)

### Some constants associated with the data set

In [ ]:
bs = 20

vocab_size = 10000

### Make a RNN class. 

In [ ]:
class handwriting_RNN(nn.Module):

    def __init__(self, hidden_size):
        super(handwriting_RNN, self).__init__()
        
        self.layer1 = nn.Embedding( vocab_size  , hidden_size  )
        self.layer2 = nn.RNN(       hidden_size , hidden_size  )
        self.layer3 = nn.Linear(    hidden_size , vocab_size   )

        
    def forward(self, word_seq, h_init ):
        
        g_seq               =   self.layer1( word_seq )  
        h_seq , h_final     =   self.layer2( g_seq , h_init )
        score_seq           =   self.layer3( h_seq )
        
        return score_seq,  h_final 

### Build the net. How many parameters in total?

In [ ]:
hidden_size=150
net = handwriting_RNN(hidden_size)
print(net)
utils.display_num_param(net)

handwriting_RNN(
  (layer1): Embedding(10000, 150)
  (layer2): RNN(150, 150)
  (layer3): Linear(in_features=150, out_features=10000, bias=True)
)
There are 3055300 (3.06 million) parameters in this neural network


### Send the weights of the networks to the GPU

In [ ]:
net = net.to(device)

### Set up manually the weights of the embedding module and Linear module

In [ ]:
net.layer1.weight.data.uniform_(-0.1, 0.1)

net.layer3.weight.data.uniform_(-0.1, 0.1)

print('')

### Choose the criterion, as well as the following important hyperparameters: 
* initial learning rate = 1
* sequence length = 35

In [ ]:
criterion = nn.CrossEntropyLoss()

my_lr = 1

seq_length = 35

### Function to evaluate the network on the test set

In [ ]:
def eval_on_test_set():

    running_loss=0
    num_batches=0    
       
    h = torch.zeros(1, bs, hidden_size)
    
    h=h.to(device)

       
    for count in range( 0 , 4120-seq_length ,  seq_length) :
               
        minibatch_data =  test_data[ count   : count+seq_length   ]
        minibatch_label = test_data[ count+1 : count+seq_length+1 ]
        
        minibatch_data=minibatch_data.to(device)
        minibatch_label=minibatch_label.to(device)
                                  
        scores, h  = net( minibatch_data, h )
        
        minibatch_label =   minibatch_label.view(  bs*seq_length ) 
        scores          =            scores.view(  bs*seq_length , vocab_size)
        
        loss = criterion(  scores ,  minibatch_label )    
        
        h=h.detach()
            
        running_loss += loss.item()
        num_batches += 1        
    
    total_loss = running_loss/num_batches 
    print('test: exp(loss) = ', math.exp(total_loss)  )

### Do 10 passes through the training set

In [ ]:
start=time.time()

for epoch in range(10):
    
    # keep the learning rate to 1 during the first 4 epochs, then divide by 1.1 at every epoch
    if epoch >= 4:
        my_lr = my_lr / 1.1
    
    # create a new optimizer and give the current learning rate.   
    optimizer=torch.optim.SGD( net.parameters() , lr=my_lr )
        
    # set the running quantities to zero at the beginning of the epoch
    running_loss=0
    num_batches=0    
       
    # set the initial h to be the zero vector
    h = torch.zeros(1, bs, hidden_size)

    # send it to the gpu    
    h=h.to(device)
    
    for count in range( 0 , 46478-seq_length ,  seq_length):
             
        # Set the gradients to zeros
        optimizer.zero_grad()
        
        # create a minibatch
        minibatch_data =  train_data[ count   : count+seq_length   ]
        minibatch_label = train_data[ count+1 : count+seq_length+1 ]        
        
        # send them to the gpu
        minibatch_data=minibatch_data.to(device)
        minibatch_label=minibatch_label.to(device)
        
        # Detach to prevent from backpropagating all the way to the beginning
        # Then tell Pytorch to start tracking all operations that will be done on h and c
        h=h.detach()
        h=h.requires_grad_()
                       
        # forward the minibatch through the net        
        scores, h  = net( minibatch_data, h )
        
        # reshape the scores and labels to huge batch of size bs*seq_length
        scores          =            scores.view(  bs*seq_length , vocab_size)  
        minibatch_label =   minibatch_label.view(  bs*seq_length )       
        
        # Compute the average of the losses of the data points in this huge batch
        loss = criterion(  scores ,  minibatch_label )
        
        # backward pass to compute dL/dR, dL/dV and dL/dW
        loss.backward()

        # do one step of stochastic gradient descent: R=R-lr(dL/dR), V=V-lr(dL/dV), ...
        utils.normalize_gradient(net)
        optimizer.step()
        
            
        # update the running loss  
        running_loss += loss.item()
        num_batches += 1
        
        
        
    # compute stats for the full training set
    total_loss = running_loss/num_batches
    elapsed = time.time()-start
    
    print('')
    print('epoch=',epoch, '\t time=', elapsed,'\t lr=', my_lr, '\t exp(loss)=',  math.exp(total_loss))
    eval_on_test_set() 

### Choose image at random from the test set and see how good/bad are the predictions

In [ ]:
label = ['Alpha', 'Beta', 'Gamma', 'Delta', 'Epsilon', 'Zeta', 'Eta', 'Theta', 'Iota', 'Kappa', 'Lambda', 'Mu', 'Nu', 'Xi', 'Omicron', 'Pi', 'Rho', 'Sigma', 'Tau', 'Upsilon', 'Phi', 'Chi', 'Psi', 'Omega', 'alpha', 'beta', 'gamma', 'delta', 'epsilon', 'zeta', 'eta', 'theta', 'iota', 'kappa', 'lambda', 'mu', 'nu', 'xi', 'omicron', 'pi', 'rho', 'sigma', 'tau', 'upsilon', 'phi', 'chi', 'psi', 'omega']

In [ ]:
# choose a picture at random
idx=randint(0, 400)
im=test_data[idx]

answer = label[test_labels[idx]]

print(answer)

# diplay the picture
utils.show(im)

# send to device, rescale, and view as a batch of 1 
im = im.to(device)
im= (im-train_mean) / train_std
im=im.view(1,64,64).unsqueeze(dim=1)

# feed it to the net and display the confidence scores
scores =  net(im) 
probs= F.softmax(scores, dim=1)
utils.show_prob_greek(probs.cpu())